# Linear regression ecc_size

In [ ]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os
import scipy
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Import data base
main_dir = '/home/mszinte/disks/meso_S/data'
project_dir = 'gaze_exp'
pp_dir = "{}/{}/derivatives/pp_data".format(main_dir, project_dir)
tsv_dir ='{}/sub-all/tsv'.format(pp_dir)
fig_dir = '{}/sub-all/figures'.format(pp_dir)
os.makedirs(fig_dir, exist_ok=True)

In [ ]:
def weighted_regression(x_reg,y_reg,weight_reg):
    """
    Function to compute regression parameter weighted by a matrix (e.g. r2 value).

    Parameters
    ----------
    x_reg : array (1D)
        x values to regress
    y_reg : array
        y values to regress
    weight_reg : array (1D) 
        weight values (0 to 1) for weighted regression

    Returns
    -------
    coef_reg : array
        regression coefficient
    intercept_reg : str
        regression intercept
    """

    from sklearn import linear_model
    import numpy as np
    
    regr = linear_model.LinearRegression()
    
    x_reg = np.array(x_reg)
    y_reg = np.array(y_reg)
    weight_reg = np.array(weight_reg)
    
    def m(x, w):
        return np.sum(x * w) / np.sum(w)

    def cov(x, y, w):
        # see https://www2.microstrategy.com/producthelp/archive/10.8/FunctionsRef/Content/FuncRef/WeightedCov__weighted_covariance_.htm
        return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

    def weighted_corr(x, y, w):
        # see https://www2.microstrategy.com/producthelp/10.4/FunctionsRef/Content/FuncRef/WeightedCorr__weighted_correlation_.htm
        return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

    
    x_reg_nan = x_reg[(~np.isnan(x_reg) & ~np.isnan(y_reg))]
    y_reg_nan = y_reg[(~np.isnan(x_reg) & ~np.isnan(y_reg))]
    weight_reg_nan = weight_reg[~np.isnan(weight_reg)]

    regr.fit(x_reg_nan.reshape(-1, 1), y_reg_nan.reshape(-1, 1),weight_reg_nan)
    coef_reg, intercept_reg = regr.coef_, regr.intercept_

    return coef_reg, intercept_reg

In [ ]:
# Load data
data = pd.read_table('{}/sub-all_prf_cf.tsv'.format(tsv_dir))
data.drop(columns=['Unnamed: 0'])

In [ ]:
# Filter data
ecc_th = [0, 20]
size_th= [0, 20]
rsq_th = [0, 1]

# Replace all data outer threshold with NaN data
data.loc[(data.prf_ecc < ecc_th[0]) | (data.prf_ecc > ecc_th[1]) | 
         (data.prf_size < size_th[0]) | (data.prf_size > size_th[1]) | 
         (data.prf_rsq_loo <=rsq_th[0])] = np.nan

data = data.dropna()

rois = pd.unique(data.roi)
mask = pd.notnull(data.subject)
subjects = pd.unique(data.subject[mask])

In [ ]:
# Define colors
roi_colors = px.colors.sequential.Sunset[:4] + px.colors.sequential.Rainbow[:]

In [ ]:
fig_height, fig_width = 400, 1200
rows, cols = 1,4
lines = [['V1', 'V2', 'V3'],['V3AB', 'LO', 'VO'],['hMT+', 'iIPS', 'sIPS'],['iPCS', 'sPCS', 'mPCS']]
for i, subject in enumerate(subjects):
    fig = make_subplots(rows=rows, cols=cols, print_grid=False)
    for l, line_label in enumerate(lines):
        for j, roi in enumerate(line_label):
            
            # Sorting best datas
            df = data.loc[(data.subject == subject) & (data.roi == roi)]
            
            # Parametring colors
            roi_color = roi_colors[j + l * 3]
            roi_color_opac = f"rgba{roi_color[3:-1]}, 0.15)"
            
            # Grouping by eccentricities
            df_grouped = df.groupby(pd.cut(df['prf_ecc'], bins=np.arange(0, 17.5, 2.5)))
            df_sorted = df.sort_values('prf_ecc')
            
            ecc_mean = np.array(df_grouped['prf_ecc'].mean())
            sd_mean = np.array(df_grouped['prf_size'].mean())
            r2_mean = np.array(df_grouped['prf_rsq_loo'].mean())
            
            # CI95 for each group of ecc
            ci = df_grouped['prf_size'].apply(lambda x: stats.t.interval(0.95, len(x)-1, loc=np.nanmean(x), scale=scipy.stats.sem(x, nan_policy='omit')))
            upper_bound = np.array(ci.apply(lambda x: x[1]))
            lower_bound = np.array(ci.apply(lambda x: x[0]))
            
            # Linear regression
            slope, intercept = weighted_regression(ecc_mean, sd_mean, r2_mean)
            slope_upper, intercept_upper = weighted_regression(ecc_mean[np.where(~np.isnan(upper_bound))], upper_bound[~np.isnan(upper_bound)], r2_mean[np.where(~np.isnan(upper_bound))])
            slope_lower, intercept_lower = weighted_regression(ecc_mean[np.where(~np.isnan(lower_bound))], lower_bound[~np.isnan(lower_bound)], r2_mean[np.where(~np.isnan(lower_bound))])
            line = slope[0][0] * np.array(df_sorted.prf_ecc) + intercept[0]
            line_upper = slope_upper[0][0] * np.array(df_sorted.prf_ecc) + intercept_upper[0]
            line_lower = slope_lower[0][0] * np.array(df_sorted.prf_ecc) + intercept_lower[0]

            fig.add_trace(go.Scatter(x=np.array(df_sorted.prf_ecc), y=line, mode='lines', name=roi, legendgroup=roi, 
                                     line=dict(color=roi_color, width=3), showlegend=False), 
                          row=1, col=l+1)

            # Error area
            fig.add_trace(go.Scatter(x=np.concatenate([df_sorted.prf_ecc, df_sorted.prf_ecc[::-1]]), 
                                     y=np.concatenate([list(line_upper), list(line_lower[::-1])]), 
                                     mode='lines', fill='toself', fillcolor=roi_color_opac, 
                                     line=dict(color=roi_color_opac, width=0), showlegend=False), 
                          row=1, col=l+1)

            # Markers
            fig.add_trace(go.Scatter(x=ecc_mean, y=sd_mean, mode='markers', 
                                     error_y=dict(type='data', array=ci.apply(lambda x: (x[1] - x[0]) / 2).tolist(), visible=True, thickness=3, width=0, color =roi_color),
                                     marker=dict(color='white', size=8, 
                                                 line=dict(color=roi_color,width=3)
                                                ), showlegend=False), 
                          row=1, col=l + 1)
            
            # Add legend
            annotation = go.layout.Annotation(x=1, y=20-j*2, text=roi, xanchor='left',
                                              showarrow=False, font=dict(color=roi_color, size=12))
            fig.add_annotation(annotation, row=1, col=l+1)
        
        # Set axis titles only for the left-most column and bottom-most row
        fig.update_yaxes(title_text='pRF size (dva)', row=1, col=1)
        fig.update_xaxes(title_text='pRF eccentricity (dva)', range=[0,15], row=1, col=l+1)
        fig.update_yaxes(range=[0,15])
        fig.update_layout(height=fig_height, width=fig_width, showlegend=False, template='simple_white')
    
    fig.show()
    fig.write_image("{}/{}_prf_size_ecc.pdf".format(fig_dir, subject)) 
    